# TAG CORPUS CONSOLIDATION

## PART 1: SIMPLE EMBEDDING

to do:   
use loss and test function to fine-tune embeddings 
add preprocessing notebook or keep it separate?

We define functions to retrieve tag data, preprocess the results, find similar embeddings, and introduce a functional test (inspired by masked-language-model training)

In [1]:
def word_preprocessing(word):
    
    import nltk
    from nltk.stem import WordNetLemmatizer
    # from nltk.corpus import wordnet
    import numpy as np
    import re
    import string
    from nltk.corpus import stopwords

    nltk.download('wordnet')
    nltk.download('stopwords')
    nltk.download('punkt')
    lemma_list = []
    word_list = []

    lemmatizer = WordNetLemmatizer()
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    stop_words = set(stopwords.words('english'))
    
    
    
    lower = word.lower()
    punct_replacer = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    rem_punct = lower.translate(punct_replacer)
    lemma = [lemmatizer.lemmatize(w) for w in nltk.word_tokenize(rem_punct)]
    rem_stop = [w for w in lemma if not w in stop_words]
    rem_digits = [re.sub('\d', '<dig>', i) for i in rem_stop]
    lemma_list.append(rem_digits)
    word_list.append(word)
    
    

In [2]:
def get_top_matches(model, test_list):
    """
    Given an embedding model and list of tags, gets most similar results based on Word2Vec embeddings 
    (model constructed in w2v_model.py).
    Runs on one row (asset) at a time.
    """
#     ref_list = []
    matches = {}
#     not_found = 0
    for lstring in test_list:
        tagset = []
        try:
            match = model.wv.most_similar(lstring)
    #         print(ls, ' : ', match)
#             ref_list.append(lstring)
            for tag in range(len(match)):
                tagset.append(match[tag][0])

#             ref_list.append(tagset)
            matches[lstring] = tagset
        except KeyError:
    #         print(ls, ' : ','NOT_FOUND')
            pass
#     print("Not found", not_found)
    return matches

# search_list = ['blue']
# get_top_matches(model, search_list)

In [3]:
def valid_prediction(test_dictionary):
    """
    Selects one random key from dictionary and determines if any values for that key
    match any other keys in the dictionary (in other words, whether the model's
    suggestion for a given tag matches any existing tags for the same asset).
    """
    import random
    rand = random.randint(0, len(test_dictionary) - 1)
    keylist = list(test_dictionary.keys())
    key = keylist[rand]
    suggestions = test_dictionary[key]
#     print(key, sugestions)
    matches = 0
    for suggestion in suggestions:
        for key in keylist:
            if suggestion == key:
#                 print("MATCH!", suggestion,  key)
                return("MATCH")
    return("NO MATCH")
    

In [4]:
##DELETE ME

# from google.cloud import bigquery
# bqclient = bigquery.Client()
# gcpd = 'w266-313114.final_project_clone'

# query = (
#     """
#     SELECT asset_id, STRING_AGG(lemma_string) as cn 
#     FROM `w266-313114.final_project_clone.raw_tags` t1
#     INNER JOIN `{}.lemmatized_tags` t2
#     on t1.tag = t2.tag
#     WHERE lemma_string != ''
#     GROUP BY 1
#     """.format(gcpd)
# )
# result = bqclient.query(query).to_dataframe()
# expanded = result['cn'].str.split(',', 51, expand=True)
# expanded = expanded.drop([51], axis=1)

We get embedding model and compute loss (for use in hyperparameter tuning)

In [5]:
#get embedding model and compute loss (for use in hyperparameter tuning)
import w2v_model
import time

for i in range(1,2):
    print('epoch:', i)
    start = time.time()
    epochs = i
    vec_size = 10
    window = 5
    vec_model = w2v_model.retrieve_model_no_id(epochs, vec_size, window)
    end = time.time()
    print('elapsed:', end - start)
    loss = vec_model.get_latest_training_loss()
    # perplexity = 2**loss
    print('loss:', loss)

/home/ejhaselden/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


epoch: 1
elapsed: 35.67230534553528
loss: 4427648.0


We compile the set of tags for each asset. For each of those tags, we then get  a list of the most similar tags based on the W2V model.

In [6]:
##to do: Make this a function with a parameter for each type of model

# get lemmatized tag df with 1 row per asset and each tag in a separate column, covert to list of lists
test_df = w2v_model.retrieve_expanded_query()
test_vals = test_df.values[0:100]

#use top_matches method to create a dictionary of related tags suggested by embedding model
asset_dicts = []
start = time.time()
for i in range(len(test_vals)):
#     print("remaining:", len(test_vals) - i)
    test_list = test_vals[i][test_vals[i] != None]
#     rate.append(test_list)
# #     print(test_list)
    top_matches = get_top_matches(vec_model, test_list)
    asset_dicts.append(top_matches)
# #     str(test_list)
end = time.time()
print("elapsed:", end - start)
# asset_dicts

elapsed: 0.8604660034179688


In [7]:
test_df[0][2]

'lil corky'

In [8]:
test_df

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,<dig><dig><dig><dig><dig><dig><dig><dig><dig> ...,original,rich fury,film industry,forehead,outerwear,spokesperson,<dig><dig><dig><dig><dig><dig><dig><dig><dig>,<dig><dig><dig><dig> <dig><dig> <dig><dig> isa...,adaptation,...,None,None,None,None,None,None,None,None,None,None
1,<dig><dig><dig><dig> anniversary <dig> word photo,arm,text,face,person,female,handwriting,black white,photography,facial expression,...,None,None,None,None,None,None,None,None,None,None
2,lil corky,lil corky bait floater,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,spring summer <dig><dig><dig><dig>,senayan city,runway,masari,ss<dig><dig>,jakarta fashion week,sjyp,jakarta fashion week <dig><dig><dig><dig>,fashion,None,...,None,None,None,None,None,None,None,None,None,None
4,<dig><dig><dig><dig><dig><dig><dig><dig><dig>,<dig><dig> <dig><dig> <dig><dig><dig><dig> <di...,circa grand opening emiller <dig><dig><dig><di...,film industry,electronic signage,electronic device,celebrity,bffulltakes ftp,led display,night,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779393,bffulltakes ftp,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
779394,bffulltakes ftp,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
779395,bffulltakes ftp,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
779396,bffulltakes ftp,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [9]:
# asset_dicts[1]

Now we have a list of dictionaries where each key is a tag for that asset and each set of values is a list of potential suggestions based on the W2V embeddings.


In [10]:
# asset_dicts[0].keys()

In [11]:
# asset_dicts[0].values()

We test the effectiveness of this suggestion set by selecting a random tag from each asset and seeing if it matches any other tag assigned to that asset. In other words, if one key matches one of another key's values.    
    
to do: (Consider averaging this over a few iterations)

In [12]:
#use valid_prediction method to determine useful suggestions
asset_results = []
for i in asset_dicts:
#     print(valid_prediction(i), i.keys())
    asset_results.append(valid_prediction(i))
print("rate of valid suggestions:", asset_results.count("MATCH")/len(asset_results))
# asset_results.count("MATCH")

rate of valid suggestions: 0.71


## PART 2: LANGUAGE MODEL    
     
code credit: https://github.com/datasci-w266/2021-summer-main/tree/master/materials/simple_lm   
to do: bring in smooting, more scoring from lm1.ipynb?  


In [13]:
# wordlist

In [14]:
import w2v_model
import time

# Standard python helper libraries.
from __future__ import print_function
from __future__ import division
import os, sys, time
import collections
import itertools

# Numerical manipulation libraries.
import numpy as np
from scipy import stats, optimize

# NLTK is the Natural Language Toolkit, and contains several language datasets
# as well as implementations of many popular NLP algorithms.
# HINT: You should look at what is available here when thinking about your project!
import nltk

# Helper libraries (see the corresponding py files in this notebook's directory).
import utils, vocabulary

# import segment

utils.require_package("tqdm")  # for nice progress bars
from tqdm import tqdm as ProgressBar

# # Bokeh for plotting.
utils.require_package("bokeh")
import bokeh.plotting as bp
from bokeh.models import HoverTool
bp.output_notebook()

Loading BokehJS ...

In [15]:
get_query = w2v_model.lm_retrieve_query()
wordlist = get_query['cn'].tolist()

In [16]:
from collections import defaultdict

def normalize_counter(c):
    """Given a dictionary of <item, counts>, return <item, fraction>."""
    total = sum(c.values())
    return {w:float(c[w])/total for w in c}

class SimpleTrigramLM(object):
    def __init__(self, words):
        """Build our simple trigram model."""
        # Raw trigram counts over the corpus. 
        # c(w | w_1 w_2) = self.counts[(w_2,w_1)][w]
        self.counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    
        # Iterate through the word stream once.
        w_1, w_2 = None, None
        for word in words:
            if w_1 is not None and w_2 is not None:
                # Increment trigram count.
                self.counts[(w_2,w_1)][word] += 1
            # Shift context along the stream of words.
            w_2 = w_1
            w_1 = word
            
        # Normalize so that for each context we have a valid probability
        # distribution (i.e. adds up to 1.0) of possible next tokens.
        self.probas = defaultdict(lambda: defaultdict(lambda: 0.0))
        for context, ctr in self.counts.items():
            self.probas[context] = normalize_counter(ctr)
            
    def next_word_proba(self, word, seq):
        """Compute p(word | seq)"""
        context = tuple(seq[-2:])  # last two words
        return self.probas[context].get(word, 0.0)
    
    def predict_next(self, seq):
        """Sample a word from the conditional distribution."""
        context = tuple(seq[-2:])  # last two words
        pc = self.probas[context]  # conditional distribution
        words, probs = zip(*pc.items())  # convert to list
        return np.random.choice(words, p=probs)
    
    def score_seq(self, seq, verbose=False):
        """Compute log probability (base 2) of the given sequence."""
        score = 0.0
        count = 0
        # Start at third word, since we need a full context.
        for i in range(2, len(seq)):
            if (seq[i] == "<s>" or seq[i] == "</s>"):
                continue  # Don't count special tokens in score.
            s = np.log2(self.next_word_proba(seq[i], seq[i-2:i]))
            score += s
            count += 1
            # DEBUG
            if verbose:
                print("log P({:s} | {:s}) = {.03f}".format(seq[i], " ".join(seq[i-2:i]), s))
        return score, count

In [17]:
split=0.8
sentences = np.array(list(wordlist), dtype=object)
fmt = (len(sentences), sum(map(len, sentences)))
print("Loaded {:,} sentences ({:g} tokens)".format(*fmt))


rng = np.random.RandomState()
rng.shuffle(sentences)  # in-place
split_idx = int(split * len(sentences))
train_sents = sentences[:split_idx]
test_sents = sentences[split_idx:]

for l in range(len(train_sents)):
    train_sents[l] = train_sents[l].split(", ")
for l in range(len(test_sents)):
    test_sents[l] = test_sents[l].split(", ")
# train_sents = train_sents.split(",")
# test_sents = test_sents.split(",")

fmt = (len(train_sents), sum(map(len, train_sents)))
print("Training set: {:,} sentences ({:,} tokens)".format(*fmt))
fmt = (len(test_sents), sum(map(len, test_sents)))
print("Test set: {:,} sentences ({:,} tokens)".format(*fmt))


Loaded 779,402 sentences (1.43624e+08 tokens)
Training set: 623,521 sentences (9,229,545 tokens)
Test set: 155,881 sentences (2,306,654 tokens)


In [18]:
train_sents

array([list(['Magenta', 'Violet', 'Purple']),
       list(['Magenta', '775640113', 'Chair', 'Pink', 'Font', 'Music', 'Purple', 'Visual Effect Lighting', 'Music Venue', 'Entertainment', 'Performing Arts', 'Arts Culture and Entertainment']),
       list(['Magenta', 'BFfulltakes_FTP', 'Performing Arts', 'Arts Culture and Entertainment', '775632493', 'Red', 'Visual Effect Lighting', 'Light', 'Entertainment', 'Performance Art']),
       ...,
       list(['Music', 'Fashion', 'Magenta', 'Performance', 'Music Artist', 'Public Event', 'Awards Ceremony', '775588770', 'Pop Music', 'Talent Show', 'Pink', 'Stage', 'Celebrities', 'Entertainment', 'BFfulltakes_FTP', 'Performing Arts', 'Arts Culture and Entertainment']),
       list(['Performance', '775571574', 'Event', 'Stage', 'Public Event', 'Entertainment', 'BFfulltakes_FTP', 'Performing Arts']),
       list(['Microphone', 'Arts Culture and Entertainment', 'ozteam', 'Art', 'Sally Capp', 'Solar Energy', 'Podium', 'Light', 'Blue', 'Spokesperson', 'E

In [19]:
import re
# Word processing functions
def canonicalize_digits(word):
    if any([c.isalpha() for c in word]): return word
    word = re.sub("\d", "DG", word)
    if word.startswith("DG"):
        word = word.replace(",", "") # remove thousands separator
    return word

def canonicalize_word(word, wordset=None, digits=True):
    word = word.lower()
    if digits:
        if (wordset != None) and (word in wordset): return word
        word = canonicalize_digits(word) # try to canonicalize numbers
    if (wordset == None) or (word in wordset):
        return word
    else:
        return constants.UNK_TOKEN

def canonicalize_words(words, **kw):
    return [canonicalize_word(word, **kw) for word in words]

In [20]:
vocab = vocabulary.Vocabulary(canonicalize_word(w) for w in ProgressBar(utils.flatten(train_sents)))
print("Train set vocabulary: %d words" % vocab.size)

100%|██████████| 9229545/9229545 [00:32<00:00, 283903.87it/s]


Train set vocabulary: 39633 words


In [21]:
def sents_to_tokens(sents):
    """Returns an flattened list of the words in the sentences, with padding for a trigram model."""
    padded_sentences = (["<s>", "<s>"] + s + ["</s>"] for s in sents)
    # This will canonicalize words, and replace anything not in vocab with <unk>
    return np.array([utils.canonicalize_word(w, wordset=vocab.wordset) 
                     for w in ProgressBar(utils.flatten(padded_sentences))], dtype=object)

train_tokens = sents_to_tokens(train_sents)
test_tokens = sents_to_tokens(test_sents)

t0 = time.time()
print("Building trigram LM...",)
lm = SimpleTrigramLM(train_tokens)
print("done in %.02f s" % (time.time() - t0))

100%|██████████| 2774297/2774297 [00:02<00:00, 945383.56it/s]


Building trigram LM...
done in 34.50 s


### Sampling Sentences    
Before we run scoring, let's look at some generated sentences from our model. We'll generate them sequentially, one token at a time:

In [22]:
def lm_predictions(l_model, tag, max_length):
        seq = ["<s>", tag]
        for i in range(max_length):
            try:
                seq.append(l_model.predict_next(seq))
            # Stop at end-of-sentence
    #         if seq[-1] == "</s>": break
            except ValueError:
                seq.append('nodata_nodata')
#         print(" ".join(seq))
#         print("[{1:d} tokens; log P(seq): {0:.02f}]".format(*lm.score_seq(seq)))
#         print("")
        seq = seq[2:]
        return seq


### Scoring  

In [23]:
log_p_data, num_real_tokens = lm.score_seq(train_tokens)
print("Train perplexity: {:.02f}".format(2**(-1*log_p_data/num_real_tokens)))

log_p_data, num_real_tokens = lm.score_seq(test_tokens)
print("Test perplexity: {:.02f}".format(2**(-1*log_p_data/num_real_tokens)))

Train perplexity: 21.26


<ipython-input-16-5e1b63fcb65f>:51: RuntimeWarning: divide by zero encountered in log2
  s = np.log2(self.next_word_proba(seq[i], seq[i-2:i]))


Test perplexity: inf


In [24]:
lengths = []
for i in asset_dicts:
    lengths.append(len(i.values()))
np.average(lengths)

15.28

In [25]:
def lm_get_top_matches(l_model, test_list):
    """
    Given a language model and list of tags, gets most similar results.
    Runs on one row (asset) at a time.
    """
#     ref_list = []
    matches = {}
#     not_found = 0
    for lstring in test_list:
#         print(lstring)
        tagset = []
        try:
            match = lm_predictions(lm, lstring, 15)
            for tag in range(len(match)):
#             for tag in range(2):
                tagset.append(match[tag])

# #             ref_list.append(tagset)
            matches[lstring] = tagset
        except KeyError:
    #         print(ls, ' : ','NOT_FOUND')
            pass
#     print("Not found", not_found)
    return matches



In [26]:
##to do: Make this a function with a parameter for each type of model

# get lemmatized tag df with 1 row per asset and each tag in a separate column, covert to list of lists
test_df = w2v_model.retrieve_expanded_query()
lm_test_vals = test_df.values[0:100]

#use top_matches method to create a dictionary of related tags suggested by embedding model
lm_asset_dicts = []
start = time.time()
for i in range(len(lm_test_vals)):
#     print("remaining:", len(test_vals) - i)
    lm_test_list = lm_test_vals[i][lm_test_vals[i] != None]
#     rate.append(test_list)
# #     print(test_list)
    lm_top_matches = lm_get_top_matches(lm, lm_test_list)
    lm_asset_dicts.append(lm_top_matches)
# #     str(test_list)
end = time.time()
print("elapsed:", end - start)
lm_asset_dicts

elapsed: 56.75792741775513


[{'<dig><dig><dig><dig><dig><dig><dig><dig><dig> <dig><dig><dig><dig> film independent spirit award vi': ['nodata_nodata',
   'nodata_nodata',
   'nodata_nodata',
   'nodata_nodata',
   'nodata_nodata',
   'nodata_nodata',
   'nodata_nodata',
   'nodata_nodata',
   'nodata_nodata',
   'nodata_nodata',
   'nodata_nodata',
   'nodata_nodata',
   'nodata_nodata',
   'nodata_nodata',
   'nodata_nodata'],
  'original': ['5.30.21 k-love awards',
   '775658984-2021 k-love fan awards',
   'gesture',
   'font',
   '5.30.21 k-love awards',
   'water',
   'orange',
   'amber',
   'arts culture and entertainment',
   '</s>',
   '<s>',
   '<s>',
   'fashion design',
   'DGDGDGDGDGDGDGDGDG',
   'eyelash'],
  'rich fury': ['original',
   'DGDGDGDGDGDGDGDGDG',
   'curtain',
   'shoe',
   'leisure',
   'goggles',
   'community',
   'facial expression',
   '775575240-2020 carousel of hope ball',
   'collage',
   'bffulltakes_ftp',
   'arts culture and entertainment',
   '</s>',
   '<s>',
   '<s>'],
  'f

In [27]:
#use valid_prediction method to determine useful suggestions
lm_asset_results = []
for i in lm_asset_dicts:
#     print(valid_prediction(i), i.keys())
    lm_asset_results.append(valid_prediction(i))
print("Rate of valid suggestions:", lm_asset_results.count("MATCH")/len(lm_asset_results))
# asset_results.count("MATCH")

Rate of valid suggestions: 0.34


## BERT  
Given that our corpus is full of unusual terms and that our "sentences" are order-agnostic, BERT's pre-trained bi-directional nature makes it a counterintuitive choice. We propose a novel application, however, in which BERT is fine-tuned on our tag corpus (onto which we impose the arbitrary order of alphabeticization).   
    
        
BERT plan:    
1. create version of reconstructed_assets with rare tags removed, each set of tags alphabetized   
2. fine-tune BERT on that <<---- this is where I'm stuck :( 
3. give BERT an unedited tag list for a given asset, with rare tags changed to [MASK]



In [28]:
#code credit: https://gist.github.com/yuchenlin/a2f42d3c4378ed7b83de65c7a2222eb2
# !pip install torchvision 
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import logging

Vanilla BERT 

In [29]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
def predict_masked_sent(text, top_k=5):
    # Tokenize input
    text = "[CLS] %s [SEP]"%text
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    # tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

        
predict_masked_sent("BERT is [MASK] at this.", top_k=5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[MASK]: 'surprised'  | weights: 0.1910516619682312
[MASK]: 'shocked'  | weights: 0.16232925653457642
[MASK]: 'amused'  | weights: 0.08754459768533707
[MASK]: 'stunned'  | weights: 0.07133634388446808
[MASK]: 'horrified'  | weights: 0.0690688043832779


Vanilla BERT given a test tag sequence

In [30]:
predict_masked_sent("[MASK], 'Lighting', 'Arts Culture and Entertainment', 'Light Fixture', 'Water', 'Chandelier', 'Ceiling Fixture', 'Ceiling', 'BFfulltakes_FTP'", top_k=5)

[MASK]: 'lighting'  | weights: 0.07484932988882065
[MASK]: 'including'  | weights: 0.02368135191500187
[MASK]: 'music'  | weights: 0.013157987967133522
[MASK]: 'architecture'  | weights: 0.00914605800062418
[MASK]: 'website'  | weights: 0.00885855220258236


In [31]:
##  TEST WITH FUNCTION

## ACTUAL SUGGESTION FUNCTION    
(filtered with rare)